In [5]:
import sys
!{sys.executable} -m pip install openpyxl

import pandas as pd
import numpy as np


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
SOURCE_FILE = "./data/dane_puste_przebiegi_cleaned.xlsx"

CITY_WITH_COUNTRY_FILE = "./data/cities_with_countries.csv"

DISTANCE_FILE = "./data/output/cities_distances_matrix.xlsx"
DISTANCE_FILE_SHEET = "Cities_Distances_Matrix"

TIME_FILE = "./data/output/cities_time_matrix.xlsx"
TIME_FILE_SHEET = "Cities_Time_Matrix"

## Load data

In [7]:
df = pd.read_excel(SOURCE_FILE)
df

,full_number,client,tractor,route,load_city,load_postal_code,load_country,unload_city,unload_postal_code,unload_country,pickup_planned,pickup_actual,delivery_planned,delivery_actual,empty_km_map,loaded_km_map,total_km_map
0,26692/2025,129.0,WGM9815L,(DE) Nürnberg - (AT) Wörgl - (DE) Nürnberg,nürnberg,90475,DE,nürnberg,90475,DE,2025-09-08 00:30:00,2025-09-08 00:46:00,2025-09-08 22:00:00,2025-09-08 22:15:00,0.000,553.013,553.013
1,27082/2025,105.0,PZ4S023,(PL) Jarosty - (CZ) Praha 5,jarosty,97-310,PL,praha 5,NaN,CZ,2025-09-08 01:10:00,2025-09-07 23:55:00,2025-09-09 03:15:00,2025-09-09 01:47:27,0.000,540.058,540.058
2,27028/2025,61.0,PZ4R993,(DE) Euskirchen - (FR) amiens,euskirchen,53881,DE,amiens,80013,FR,2025-09-08 05:00:00,2025-09-08 16:32:00,2025-09-08 15:00:00,2025-09-09 11:16:00,0.000,409.626,409.626
3,27026/2025,47.0,WGM8283K,(CZ) Modřice - (CZ) Pelhřimov - (CZ) VRSKMAN,modřice,66442,CZ,vrskman,43111,CZ,2025-09-08 05:30:00,2025-09-08 05:48:28,2025-09-08 12:30:00,2025-09-08 14:04:21,318.425,351.447,669.872
4,27049/2025,105.0,WGM5118H,(PL) Chlastawa - (DE) Berlin,chlastawa,66-210,PL,berlin,13597,DE,2025-09-08 06:00:00,2025-09-09 00:34:00,2025-09-09 06:00:00,2025-09-09 05:17:00,0.789,216.089,216.878
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1638,29273/2025,61.0,WGM9817L,(FR) amiens - (DE) Langenbach,amiens,80013,FR,langenbach,85416,DE,2025-09-27 10:00:00,2025-09-27 13:31:00,2025-09-30 15:30:00,NaT,0.000,913.536,913.536
1639,29398/2025,105.0,WGM5121H,(PL) Osieczna - (CZ) Praha - CERNY MOST,osieczna,64-113,PL,praha - cerny most,19800,CZ,2025-09-27 10:00:00,2025-09-27 20:10:04,2025-09-29 20:45:00,NaT,0.973,372.595,373.568
1640,29444/2025,90.0,PZ911JJ,(PL) OSIECZNA - (PL) Wiskitki,osieczna,64-113,PL,wiskitki,NaN,PL,2025-09-27 12:00:00,2025-09-26 14:09:12,2025-09-27 17:00:00,2025-09-27 20:12:12,0.000,311.320,311.320
1641,29515/2025,61.0,PO2PY72,(PL) OSIECZNA - (PL) Sochaczew,osieczna,64-113,PL,sochaczew,96-500,PL,2025-09-27 14:00:00,NaT,2025-10-01 03:00:00,NaT,0.000,316.000,316.000


## Cities distance & time matrix

### Cites with country code

In [8]:
# Create a list of (city, country) pairs from load and unload columns
load_pairs = df[["load_city", "load_country"]].apply(tuple, axis=1)
unload_pairs = df[["unload_city", "unload_country"]].apply(tuple, axis=1)

# Merge both lists and remove duplicates and NaN values
city_country_pairs = load_pairs.tolist() + unload_pairs.tolist()
city_country_pairs = [pair for pair in city_country_pairs if pd.notna(pair[0]) and pd.notna(pair[1])]

# Unique and sorted pairs
city_country_pairs = sorted(set(city_country_pairs))

print(f"Number of unique city / country pairs: {len(city_country_pairs)}")

city_country_pairs_df = pd.DataFrame(city_country_pairs, columns=["city", "country_code"])
city_country_pairs_df.to_csv(CITY_WITH_COUNTRY_FILE, index=False)
print(f"City / country pairs saved to: {CITY_WITH_COUNTRY_FILE}")


Number of unique city / country pairs: 702
City / country pairs saved to: ./data/cities_with_countries.csv


### Matrix of cities

In [9]:
unique_cities = [city for city, country in city_country_pairs]

In [10]:
matrix = pd.DataFrame(
    data=0,
    index=unique_cities,
    columns=unique_cities
)

In [11]:
matrix

,aalen,abanilla,ablis,abrera,agnadello,alicante,alizay,altfeld,alzenau,alzey,...,öhringen,ühlingen-birkendorf,łódź,říčany,środa śląska gmina,świebodzice,świebodzin,święciechowa,żary,žirovnice
aalen,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
abanilla,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ablis,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
abrera,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
agnadello,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
świebodzice,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
świebodzin,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
święciechowa,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
żary,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
matrix.to_excel(DISTANCE_FILE, sheet_name=DISTANCE_FILE_SHEET)
matrix.to_excel(TIME_FILE, sheet_name=TIME_FILE_SHEET)

print(f"Matrix saved to: {DISTANCE_FILE} & {TIME_FILE}")

Matrix saved to: ./data/output/cities_distances_matrix.xlsx & ./data/output/cities_time_matrix.xlsx
